Zadanie:
Zbuduj model sieci konwolucyjnej (CNN) w Keras, który będzie w stanie rozróżnić obrazy kotów od obrazów psów. Model powinien korzystać z modelu sekwencyjnego i zawierać następujące warstwy:

- Warstwę konwolucyjną (Conv2D) z 32 filtrami o rozmiarze 3x3, funkcją aktywacji ReLU, i odpowiednim kształtem wejściowym dla obrazów (np. 64x64x3).
- Warstwę Max Pooling (MaxPooling2D) o rozmiarze 2x2.
- Drugą warstwę konwolucyjną (Conv2D) z 64 filtrami o rozmiarze 3x3 i funkcją aktywacji ReLU.
- Drugą warstwę Max Pooling (MaxPooling2D) o rozmiarze 2x2.
- Warstwę spłaszczającą (Flatten) do przekształcenia danych 2D na format 1D.
- Warstwę gęstą (Dense) z 64 jednostkami i funkcją aktywacji ReLU.
- Warstwę wyjściową (Dense) z jedną jednostką i funkcją aktywacji sigmoid, która będzie wskazywać prawdopodobieństwo przynależności obrazu do jednej z klas (kot lub pies).

Dodatkowo, zaimplementuj kompilację modelu z optymalizatorem (np. Adam), funkcją straty binary crossentropy, oraz metryką 'accuracy'. Przygotuj krótki opis, co każda z warstw robi i dlaczego została wybrana.

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Inicjalizacja modelu sekwencyjnego
model = Sequential()

# Dodanie warstw do modelu
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(64, 64, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))

# Kompilacja modelu
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Opisz każdą warstwę używając komentarzy w kodzie


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Model

# Wejście modelu
input_img = Input(shape=(64, 64, 3))

# Pierwsza warstwa konwolucyjna
x = Conv2D(32, (3, 3), activation='relu')(input_img)
x = MaxPooling2D((2, 2))(x)

# Druga warstwa konwolucyjna
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)

# Trzecia warstwa konwolucyjna
x = Conv2D(128, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)

# Spłaszczenie i pełna połączona warstwa
x = Flatten()(x)
x = Dense(512, activation='relu')(x)

# Dropout
x = Dropout(0.5)(x)

# Warstwa wyjściowa
output = Dense(1, activation='sigmoid')(x)

# Tworzenie modelu
model = Model(inputs=input_img, outputs=output)

# Kompilacja modelu
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Podsumowanie modelu
model.summary()

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

# Przygotowanie danych (przykładowe ścieżki do katalogów)
train_directory = 'D:\\MlTutor\\SieciNeuronowe\\02_cnn\\cats_dogs\\train'
validation_directory = 'D:\\MlTutor\\SieciNeuronowe\\02_cnn\\cats_dogs\\test'

# Augmentacja danych dla zestawu treningowego
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Augmentacja danych jest zazwyczaj stosowana tylko dla zestawu treningowego.
validation_datagen = ImageDataGenerator(rescale=1./255)

# Przepływ danych treningowych i walidacyjnych
train_generator = train_datagen.flow_from_directory(
    train_directory,
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_directory,
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)

# Callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True),
    ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, verbose=1, mode='auto', min_lr=0.00001)
]

# Uczenie modelu
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples//train_generator.batch_size,
    epochs=50,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples//validation_generator.batch_size,
    callbacks=callbacks
)


A jak to wygląda? 

In [ ]:
import matplotlib.pyplot as plt

# Pobieranie wsadu obrazów i etykiet
for images_batch, labels_batch in train_generator:
    break  # Zatrzymujemy się po pierwszym wsadzie

# Definiowanie rozmiaru figury dla wykresu
plt.figure(figsize=(10, 10))

# Wyświetlanie pierwszych 9 obrazów z wsadu
for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images_batch[i])
    plt.title(int(labels_batch[i]))
    plt.axis("off")

# Wyświetlanie wykresu
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import keras.utils as image
from keras.preprocessing.image import ImageDataGenerator

# Załaduj obraz
img_path = 'D:\\MlTutor\\SieciNeuronowe\\02_cnn\\cats_dogs\\kitku.jpg'
img = image.load_img(img_path, target_size=(64, 64))  # Asumujemy, że używasz obrazów 64x64
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)

# Stwórz generator z augmentacją
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Wygeneruj obrazy
it = datagen.flow(img_array, batch_size=1)

# Wyświetl oryginalny obraz
plt.figure(figsize=(6, 3))

plt.subplot(1, 2, 1)
plt.imshow(image.array_to_img(img_array[0]))
plt.title('Original Image')
plt.axis('off')

# Wygeneruj i wyświetl transformowany obraz
plt.subplot(1, 2, 2)
batch = it.next()
transformed_img = image.array_to_img(batch[0])
plt.imshow(transformed_img)
plt.title('Transformed Image')
plt.axis('off')

plt.show()


Transfer learning

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

train_directory = 'D:\\MlTutor\\SieciNeuronowe\\02_cnn\\cats_dogs\\train'
validation_directory = 'D:\\MlTutor\\SieciNeuronowe\\02_cnn\\cats_dogs\\test'

# Wczytanie modelu VGG16 bez włączonych górnych warstw
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Zamrożenie warstw bazowego modelu
for layer in base_model.layers:
    layer.trainable = False

# Tworzenie nowego modelu na górze wstępnie wytrenowanego modelu
model = Sequential([
    base_model,
    Flatten(),
    Dense(1024, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')  # 2 klasy: koty i psy
])

# Kompilacja modelu
model.compile(optimizer=Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

# Stworzenie generatorów danych z użyciem funkcji preprocess_input z VGG16
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# Przygotowanie generatorów danych treningowych i walidacyjnych
train_generator = train_datagen.flow_from_directory(
    train_directory,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = test_datagen.flow_from_directory(
    validation_directory,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Trenowanie modelu
history = model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

A co z hyper parameter fine tunning?